# The Simplex algotithm

A simple implementation for the primal formulation of the simplex algorithm.

## Formulation of a simple problem

In [1]:
import numpy as np
from fractions import Fraction

In [2]:
_A = np.array(
    [
        [1, 3, 1, 1, 0, 0, 0],
        [-1, 0, 3, 0, 1, 0, 0],
        [2, -1, 2, 0, 0, 1, 0],
        [2, 3, -1, 0, 0, 0, 1]
    ],
    dtype=np.int64
)

_A

array([[ 1,  3,  1,  1,  0,  0,  0],
       [-1,  0,  3,  0,  1,  0,  0],
       [ 2, -1,  2,  0,  0,  1,  0],
       [ 2,  3, -1,  0,  0,  0,  1]])

In [3]:
_p = np.array([ 0, 0, 0, 3, 2, 4, 2], dtype=np.int64)
_p

array([0, 0, 0, 3, 2, 4, 2])

In [4]:
_z = np.array([5, 5, 3, 0, 0, 0, 0], dtype=np.int64)
_z

array([5, 5, 3, 0, 0, 0, 0])

In [5]:
r = Fraction(0)
r

Fraction(0, 1)

In [6]:
from mcf_simplex_analyzer.load_instance import FractionArray

In [7]:
A = FractionArray(_A, np.ones_like(_A))
A

FractionArray(numerators=array([[ 1,  3,  1,  1,  0,  0,  0],
       [-1,  0,  3,  0,  1,  0,  0],
       [ 2, -1,  2,  0,  0,  1,  0],
       [ 2,  3, -1,  0,  0,  0,  1]]), denominators=array([[1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1]]))

In [8]:
p = FractionArray(_p, np.ones_like(_p))
p

FractionArray(numerators=array([0, 0, 0, 3, 2, 4, 2]), denominators=array([1, 1, 1, 1, 1, 1, 1]))

In [9]:
base = np.array([0, 0, 0, 1, 1, 1, 1], dtype=bool)
base_ind = np.where(base)[0]
base_ind

array([3, 4, 5, 6])

In [10]:
z = FractionArray(_z, np.ones_like(_z))
z

FractionArray(numerators=array([5, 5, 3, 0, 0, 0, 0]), denominators=array([1, 1, 1, 1, 1, 1, 1]))

## Entering variable (Dantzig)

In [52]:
lcm_z = np.lcm.reduce(z.denominators)
entering = np.argmax((lcm_z // z.denominators) * z.numerators)
entering

0

## Leaving variable (Dantzig)

In [12]:
lcm_A = np.lcm.reduce(A.denominators[..., entering])
np.amin(A.numerators[..., entering] * lcm_A)

array([ 1, -1,  2,  2])

In [66]:
bound_nominators = p.numerators[base] * A.denominators[..., entering]
bound_denominators = p.denominators[base] *  A.numerators[..., entering]

sign_nominators = bound_nominators < 0
sign_denominators = bound_denominators < 0
sign = np.logical_xor(sign_nominators, sign_denominators)

lcm_bound = np.lcm.reduce(bound_denominators)
bounds = bound_nominators * (lcm_bound // bound_denominators)

sign, bounds

(array([False,  True, False, False]), array([ 6, -4,  4,  2]))

In [59]:
valid = np.where(~sign)[0]
valid

array([0, 2, 3])

In [60]:
leaving_base_ind = valid[bounds[valid].argmin()]
leaving_base_ind

3

In [67]:
leaving = base_ind[leaving_base_ind]
leaving

6

In [68]:
new_r = r + Fraction(bounds[leaving_base_ind], lcm_bound)
new_r

Fraction(1, 1)